# Trading EDA

To install conda and binance packages to this notebook uncomment the code below

In [175]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
#%pip install binance-connector==1.13.0

Init binance client

In [167]:
from binance.spot import Spot as Client
import os
import sys
import yaml

# Load 
def load_config():
    """
    Load config from cfg folder respecting the order: defaults, app.yaml, environment vars
    """
    # Defaults
    cfg_dir="./../biml/cfg"
    default_cfg_path = f"{cfg_dir}/app-defaults.yaml"
    with open(default_cfg_path, "r") as appdefaults:
        config = yaml.safe_load(appdefaults)

    # Custom config, should contain custom information,
    cfg_path = f"{cfg_dir}/app.yaml"
    if os.path.exists(cfg_path):
        with open(cfg_path) as app:
            config.update(yaml.safe_load(app))
    else:
        sys.exit(
            f"Config {cfg_path} not found. Please copy cfg/app-defaults.yaml to {cfg_path} "
            f"and update connection info there.")

    # Enviroment variabless
    config.update(os.environ)
    return config


# Create binance client
config=load_config()
key,secret,url=config["biml.connector.key"],config["biml.connector.secret"],config["biml.connector.url"]
client = Client(key=key, secret=secret, base_url=url, timeout=10)


Get opened orders from binance

In [169]:
# Get orders
symbol="BTCUSDT"
opened_orders_dict = client.get_open_orders(symbol)
orders=pd.DataFrame(opened_orders_dict)
if not orders.empty: orders=orders.astype({'time': 'datetime64[ms]','updateTime':'datetime64[ms]'}).set_index('time',drop=False)
orders.head(100)

""


Get trades from Binance

In [27]:
import pandas as pd

#mytrades_dict=[{'symbol': 'BTCUSDT', 'id': 882731, 'orderId': 1957295, 'orderListId': 367, 'price': '21900.00000000', 'qty': '0.00100000', 'quoteQty': '21.90000000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1655261128431, 'isBuyer': False, 'isMaker': True, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 907300, 'orderId': 2003764, 'orderListId': -1, 'price': '20860.34000000', 'qty': '0.00100000', 'quoteQty': '20.86034000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'time': 1655271037368, 'isBuyer': True, 'isMaker': False, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2860170, 'orderId': 6104147, 'orderListId': -1, 'price': '21481.31000000', 'qty': '0.00100000', 'quoteQty': '21.48131000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1656149861641, 'isBuyer': False, 'isMaker': True, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2901517, 'orderId': 6214259, 'orderListId': -1, 'price': '20981.74000000', 'qty': '0.00100000', 'quoteQty': '20.98174000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'time': 1656172361645, 'isBuyer': True, 'isMaker': False, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2901645, 'orderId': 6214506, 'orderListId': -1, 'price': '21000.00000000', 'qty': '0.00100000', 'quoteQty': '21.00000000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1656172400009, 'isBuyer': False, 'isMaker': False, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2901714, 'orderId': 6214673, 'orderListId': -1, 'price': '20970.56000000', 'qty': '0.00100000', 'quoteQty': '20.97056000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'time': 1656172432877, 'isBuyer': True, 'isMaker': False, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2987743, 'orderId': 6486737, 'orderListId': -1, 'price': '21373.90000000', 'qty': '0.00100000', 'quoteQty': '21.37390000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1656229949510, 'isBuyer': False, 'isMaker': True, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 2987760, 'orderId': 6486675, 'orderListId': -1, 'price': '21377.89000000', 'qty': '0.00100000', 'quoteQty': '21.37789000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'time': 1656229952525, 'isBuyer': False, 'isMaker': True, 'isBestMatch': True}, {'symbol': 'BTCUSDT', 'id': 3063166, 'orderId': 6717004, 'orderListId': -1, 'price': '21336.61000000', 'qty': '0.00200000', 'quoteQty': '42.67322000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'time': 1656278728249, 'isBuyer': True, 'isMaker': False, 'isBestMatch': True}]
symbol="BTCUSDT"
mytrades_dict = client.my_trades(symbol)

# Preprocess trades df
mytrades=pd.DataFrame(mytrades_dict)
mytrades=mytrades.astype({'time': 'datetime64[ms]'}).set_index('time',drop=False)
mytrades.qty=np.where(mytrades.isBuyer==False,mytrades.qty.astype('float')*-1,mytrades.qty.astype('float'))
mytrades["qtycumsum"]=mytrades["qty"].cumsum()
mytrades.tail()

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch,qtycumsum
time,,,,,,,,,,,,,,
2022-06-26 07:52:29.510,BTCUSDT,2987743,6486737,-1,21373.90000000,-0.001,21.37390000,0.00000000,USDT,2022-06-26 07:52:29.510,False,True,True,-0.001
2022-06-26 07:52:32.525,BTCUSDT,2987760,6486675,-1,21377.89000000,-0.001,21.37789000,0.00000000,USDT,2022-06-26 07:52:32.525,False,True,True,-0.002
2022-06-26 21:25:28.249,BTCUSDT,3063166,6717004,-1,21336.61000000,0.002,42.67322000,0.00000000,BTC,2022-06-26 21:25:28.249,True,False,True,0.000
2022-06-26 22:41:09.277,BTCUSDT,3071166,6739104,-1,21028.81000000,0.001,21.02881000,0.00000000,BTC,2022-06-26 22:41:09.277,True,False,True,0.001
2022-06-27 13:43:57.067,BTCUSDT,3160872,6739109,-1,20609.98000000,0.001,20.60998000,0.00000000,BTC,2022-06-27 13:43:57.067,True,True,True,0.002


Get candles from local history starting from earliest trade date ending by latest trade date.

In [173]:
import os
import glob
import pandas as pd
from datetime import timedelta,date

def read_candles(start_date_inc, end_date_inc):
    """ Read days from start to end from local history"""
    symbol="BTCUSDT"    
    data_dir=f"./../data/{symbol}"
    print(f"start date:{start_date_inc}, end date:{end_date_inc}, all inclusive")
    candles = pd.DataFrame()
    cur_candles_list=[]
    for n in range(int ((end_date_inc - start_date_inc).days)+1):
        cur_date = start_date_inc + timedelta(n)    
        csv_path=f"{data_dir}/{cur_date}_{symbol}_1m.csv"        
        #print(f"Read {cur_date} candles from {csv_path}")
        cur_candles=pd.read_csv(csv_path,index_col="close_time")[["open","high","low","close","vol"]]
        cur_candles_list.append(cur_candles)
    candles = pd.concat(cur_candles_list)
    return candles

def clean_candles(df):
    """ Remove outliers """
    q_low = df["low"].quantile(0.01)
    q_high  = df["high"].quantile(0.99)
    df=df[df["high"]< q_high]
    df=df[df["low"] > q_low ]
    return df

# Read candles for my_trades interval
candles = read_candles(mytrades.index.min().date(), mytrades.index.max().date())
candles = clean_candles(candles)
candles.tail()


start date:2022-06-15, end date:2022-06-27, all inclusive


,open,high,low,close,vol
close_time,,,,,
2022-06-27 23:55:59.999,20734.14,20735.89,20731.78,20734.56,2.906291
2022-06-27 23:56:59.999,20733.21,20733.21,20722.85,20732.80,4.947727
2022-06-27 23:57:59.999,20732.94,20734.61,20725.88,20734.61,4.704985
2022-06-27 23:58:59.999,20734.86,20735.36,20716.69,20718.45,5.880414
2022-06-27 23:59:59.999,20722.90,20727.64,20715.16,20727.64,5.797677


## EDA

### Look at trades quantity cumulative sum

In [50]:
df=mytrades.copy()[["symbol","price","qty","qtycumsum"]]
df.price=df.price.astype("float")
df["pricedelta"]=df["price"].diff()
df.tail(1000)

,symbol,price,qty,qtycumsum,pricedelta
time,,,,,
2022-06-15 02:45:28.431,BTCUSDT,21900.00,-0.001,-0.001,NaN
2022-06-15 05:30:37.368,BTCUSDT,20860.34,0.001,0.000,-1039.66
2022-06-25 09:37:41.641,BTCUSDT,21481.31,-0.001,-0.001,620.97
2022-06-25 15:52:41.645,BTCUSDT,20981.74,0.001,0.000,-499.57
2022-06-25 15:53:20.009,BTCUSDT,21000.00,-0.001,-0.001,18.26
2022-06-25 15:53:52.877,BTCUSDT,20970.56,0.001,0.000,-29.44
2022-06-26 07:52:29.510,BTCUSDT,21373.90,-0.001,-0.001,403.34
2022-06-26 07:52:32.525,BTCUSDT,21377.89,-0.001,-0.002,3.99
2022-06-26 21:25:28.249,BTCUSDT,21336.61,0.002,0.000,-41.28


## Look at trades on candlestick chart

In [40]:
# Install a conda package in the current Jupyter kernel
#%conda install -c plotly plotly=5.9.0
# Draw candlechart
import plotly as py
from plotly import graph_objects as go
import pandas as pd

df=candles.copy()
#df=clean_candles(df)
fig = go.Figure(data=[ \
                    go.Candlestick(\
                        x=df.index,\
                        open=df['open'],\
                        high=df['high'],\
                        low=df['low'],\
                        close=df['close'])
                     ])

# Customize ay to distribute captions by y so they do not overlap by x
ay,aymax,aystep=0,50,10
# Annotate buy, sell on the chart
for orderitem in mytrades[mytrades.index>df.index.min()].iterrows():
    ay=(ay+aystep)%aymax
    order=orderitem[1]
    color='green' if order['isBuyer'] else 'red'
    text_short='buy' if order['isBuyer'] else 'sell'
    text=f"{order['time']} {text_short} at {order['price']}"
    fig.add_annotation(x=order['time'], \
                       y=order['price'],\
                       text=text_short,\
                       hovertext=text,\
                       showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                       font={'color':color}
                      )
fig.show()